In [ ]:
import os
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn import preprocessing

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn import metrics

sns.set_style("whitegrid")

In [ ]:
# import merged df
df = pd.read_csv('merged_df.csv',
    index_col=False,
    low_memory=False
)
# drop unncessary columns
df = df.drop('Unnamed: 0', axis=1)
df.head()

In [ ]:
# check NA sum for each column
for col in df.columns:
    print(col, df[col].isna().sum())

In [ ]:
# fill NA values with 0 (alive) for 'deceased' column
df['deceased'] = df['deceased'].fillna(value=0)

# drop unwanted columns
df = df.drop(['death_date_x', 
              'death_date_y', 
              'American Indian or Alaska Native_y',
              'Asian_y', 
              'Unknown_y'], 
             axis=1)

df.rename(columns={'Values_Average':'0'}, inplace=True)

In [ ]:
# drop all columns with >= 55% NA
df2 = df.loc[:, df.isnull().mean() < 0.55]
df2.head()

### Imputation of Missing Values

In [ ]:
# impute with median values
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values='NaN', strategy='median')

df_imp = df2.values
df_imp = imp.fit_transform(df_imp)
# rebuild df using index and column of original df
df_imp = pd.DataFrame(data=df_imp, index=df2.index, columns=df2.columns)
df_imp.head()

In [ ]:
# check NA sum for each column
for col in df_imp.columns:
    print(col, df_imp[col].isna().sum())

### Models

In [ ]:
# set up features and labels for train/test
features = df_new_imp.drop(['deceased', 'person_id'], axis=1) # drop only 'new bone formation' col
labels = df_new_imp['deceased'] # single column with outcome of interest

plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

# barplot of outcome of interest
# in our sample, deceased and alive are roughly equal
sns.countplot(x=labels, data=df2, palette='Purples')
plt.show()
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/Distribution.png', bbox_inches='tight')

# train and test split 80:20
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=42)
print(features_train.shape)
print(features_test.shape)
print(labels_train.shape)
print(labels_test.shape)

#### Logistic Regression

In [ ]:
# logistic regression modeling
lr = LogisticRegression(max_iter=1000)
lr.fit(features_train, labels_train)
lr_pred = lr.predict(features_test)
print('Accuracy of logistic regression classifier on test set: {:.3f}'.format(lr.score(features_test, labels_test)))

In [ ]:
# confusion matrix
cm = confusion_matrix(labels_test, lr_pred)
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

sns.heatmap(cm, cmap='Purples', annot=True, fmt='d', cbar=False, annot_kws = {"size": 15})
plt.title('Logistic Regression \nAccuracy:{0:.3f}'.format(accuracy_score(labels_test, lr_pred)), fontsize=15)
plt.ylabel('True label', fontsize=15)
plt.xlabel('Predicted label', fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/LR_CM.png', bbox_inches='tight')

In [ ]:
# ROC Curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

sns.set_style("whitegrid")

logit_roc_auc = roc_auc_score(labels_test, lr.predict(features_test))
fpr, tpr, thresholds = roc_curve(labels_test, lr.predict_proba(features_test)[:,1])
plt.figure(figsize=(16,12))
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.3f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('Receiver operating characteristic', fontsize=15)
plt.legend(loc="lower right", fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/LR_ROC.png', bbox_inches='tight')

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10000, 
                            bootstrap = True,
                            max_features = 'sqrt', 
                            oob_score=True)

rf.fit(features_train, labels_train)
rf_pred = rf.predict(features_test)
rf_probs = rf.predict_proba(features_test)[:, 1]

from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(labels_test, rf_pred))

In [ ]:
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")
cm = confusion_matrix(labels_test, rf_pred)

sns.heatmap(cm, cmap='Purples', annot=True, fmt='d', cbar=False, annot_kws = {"size": 15})
plt.title('Random Forest \nAccuracy:{0:.3f}'.format(accuracy_score(labels_test, rf_pred)), fontsize=15)
plt.ylabel('True label', fontsize=15)
plt.xlabel('Predicted label', fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/RF_CM.png', bbox_inches='tight')

In [ ]:
# ROC Curve
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

rf_roc = roc_auc_score(labels_test, rf_probs)
fpr, tpr, thresholds = roc_curve(labels_test, rf.predict_proba(features_test)[:,1])
plt.figure(figsize=(16,12))
plt.plot(fpr, tpr, label='Random Forest (area = %0.3f)' % rf_roc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('Receiver operating characteristic', fontsize=15)
plt.legend(loc="lower right", fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/RF_ROC.png', bbox_inches='tight')

#### Random Forest Feature Importance

In [ ]:
# feature importance from random forest
feature_imp = pd.Series(rf.feature_importances_, index=features_train.columns).sort_values(ascending=False)
feature_imp

features = feature_imp.iloc[0:5,]

plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

sns.barplot(x=features, y=features.index, palette='Purples_r')
plt.xlabel('Feature Importance Score', fontsize=15)
plt.yticks(fontsize=12)
plt.title("Feature Importance", fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/RF_Features.png', bbox_inches='tight')

In [ ]:
# select top 20 features and re-test random forest
top_20 = feature_imp.iloc[0:20,]
death = df_imp['deceased']
df_topfeatures = df_imp.loc[:,top_20.index]

X = df_topfeatures # drop only 'new bone formation' col
y = df_imp['deceased'] # single column with outcome of interest

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# random forest on top 25 features
rf = RandomForestClassifier(n_estimators=10000, 
                            bootstrap = True,
                            max_features = 'sqrt', 
                            oob_score=True)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_probs = rf.predict_proba(X_test)[:, 1]

from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, rf_pred))

In [ ]:
cm = confusion_matrix(y_test, rf_pred)
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

sns.heatmap(cm, cmap='Purples', annot=True, fmt='d', cbar=False, annot_kws = {"size": 15})
plt.title('Random Forest \nAccuracy:{0:.3f}'.format(accuracy_score(y_test, rf_pred)), fontsize=15)
plt.ylabel('True label', fontsize=15)
plt.xlabel('Predicted label', fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/RF_Features_CM.png', bbox_inches='tight')

In [ ]:
rf_roc = roc_auc_score(y_test, rf_probs)
fpr, tpr, thresholds = roc_curve(y_test, rf.predict_proba(X_test)[:,1])
plt.figure(figsize=(16,12))
plt.plot(fpr, tpr, label='Random Forest (area = %0.3f)' % rf_roc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('Receiver operating characteristic', fontsize=15)
plt.legend(loc="lower right", fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/RF_Features_ROC.png', bbox_inches='tight')

#### Support Vector Machine

In [ ]:
# SVM with Linear Kernel
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

svm = LinearSVC(random_state=0, tol=1e-5, max_iter=2000)
svm.fit(features_train, labels_train)
clf = CalibratedClassifierCV(svm)
clf.fit(features_train, labels_train)
svm_pred = clf.predict(features_test)
svm_probs = clf.predict_proba(features_test)[:, 1]
print("SVM Linear Accuracy:",metrics.accuracy_score(labels_test, svm_pred))

In [ ]:
svm_roc = roc_auc_score(labels_test, svm_probs)
fpr, tpr, thresholds = roc_curve(labels_test, clf.predict_proba(features_test)[:,1])
plt.figure(figsize=(16,12))
plt.plot(fpr, tpr, label='SVM Linear Kernel (AUC = %0.3f)' % svm_roc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('Receiver operating characteristic', fontsize=15)
plt.legend(loc="lower right", fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_Linear_ROC.png', bbox_inches='tight')

In [ ]:
cm = confusion_matrix(labels_test, svm_pred)
plt.figure(figsize=(16,12))

sns.heatmap(cm, cmap='Purples', annot=True, fmt='d', cbar=False, annot_kws = {"size": 15})
plt.title('SVM Linear Kernel \nAccuracy:{0:.3f}'.format(accuracy_score(labels_test, svm_pred)), fontsize=15)
plt.ylabel('True label', fontsize=15)
plt.xlabel('Predicted label', fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_Linear_CM.png', bbox_inches='tight')

#### SVM Feature Selection

In [ ]:
# extract coefficients from SVM to list
coef = svm.coef_

coef = [y for x in coef for y in x]
coef

In [ ]:
# match features to coefficients
features_svm = features.columns.to_series(index=range(len(features.columns)))
coef = pd.Series(coef, index=range(len(features.columns)))
features_svm.name = 'Features'
coef.name = 'Coefficient'

# concatenate features and coefficients to df
feature_coef = pd.concat([features_svm, coef], axis=1)
feature_coef.head()

# sort by coefficients
sort_coef = feature_coef.sort_values(by='Coefficient').reset_index(drop=True)
sort_coef.head()

In [ ]:
# save top 10 and bottom 10 features
top_features = sort_coef.head(10).append(sort_coef.tail(10))

# barplot of features
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

sns.barplot(x='Coefficient', y='Features', data=top_features, palette='RdBu', order=top_features['Features'], orient='h')
plt.xlabel('Coefficient', fontsize=15)
plt.ylabel('')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('SVM Features', fontsize=20)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_Features.png', bbox_inches='tight')

#### SVM with Linear Kernel Using Top Features

In [ ]:
svm_features = top_features['Features']

X = features.loc[:,svm_features]
y = df_new_imp['deceased'] # single column with outcome of interest

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

svm = LinearSVC(random_state=0, tol=1e-5)
svm.fit(X_train, y_train)
clf = CalibratedClassifierCV(svm)
clf.fit(X_train, y_train)
svm_pred = clf.predict(X_test)
svm_probs = clf.predict_proba(X_test)[:, 1]
print("SVM Linear Accuracy:",metrics.accuracy_score(y_test, svm_pred))

In [ ]:
svm_roc = roc_auc_score(y_test, svm_probs)
fpr, tpr, thresholds = roc_curve(y_test, clf.predict_proba(X_test)[:,1])
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

plt.plot(fpr, tpr, label='SVM Linear Kernel (AUC = %0.3f)' % svm_roc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('Receiver operating characteristic', fontsize=15)
plt.legend(loc="lower right", fontsize=15)
plt.show()
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_Linear_ROC.png', bbox_inches='tight')

cm = confusion_matrix(labels_test, svm_pred)
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

sns.heatmap(cm, cmap='Purples', annot=True, fmt='d', cbar=False, annot_kws = {"size": 15})
plt.title('SVM Linear Kernel \nAccuracy:{0:.3f}'.format(accuracy_score(y_test, svm_pred)), fontsize=15)
plt.ylabel('True label', fontsize=15)
plt.xlabel('Predicted label', fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_Linear_CM.png', bbox_inches='tight')

#### SVM with RBF Kernel

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.svm import SVC

clf = OneVsRestClassifier(SVC(kernel='rbf', probability=True, class_weight=None), n_jobs=-1)
clf.fit(features_train, labels_train)
svm_pred = clf.predict(features_test)
svm_probs = clf.predict_proba(features_test)[:, 1]
print("SVM RBF Accuracy:",metrics.accuracy_score(labels_test, svm_pred))

In [ ]:
svm_roc = roc_auc_score(labels_test, svm_probs)
fpr, tpr, thresholds = roc_curve(labels_test, clf.predict_proba(features_test)[:,1])
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

plt.plot(fpr, tpr, label='SVM RBF Kernel (AUC = %0.3f)' % svm_roc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('Receiver operating characteristic', fontsize=15)
plt.legend(loc="lower right", fontsize=15)
plt.show()
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_RBF_ROC.png', bbox_inches='tight')

cm = confusion_matrix(labels_test, svm_pred)
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

sns.heatmap(cm, cmap='Purples', annot=True, fmt='d', cbar=False, annot_kws = {"size": 15})
plt.title('SVM RBF Kernel \nAccuracy:{0:.3f}'.format(accuracy_score(labels_test, svm_pred)), fontsize=15)
plt.ylabel('True label', fontsize=15)
plt.xlabel('Predicted label', fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_RBF_CM.png', bbox_inches='tight')

#### SVM with Polynomial Kernel

In [ ]:
clf = OneVsRestClassifier(SVC(kernel='poly', degree=3, probability=True), n_jobs=-1)                                                                                    
clf.fit(features_train, labels_train)
svm_pred = clf.predict(features_test)
svm_probs = clf.predict_proba(features_test)[:, 1]
print("SVM Polynomial Accuracy:",metrics.accuracy_score(labels_test, svm_pred))

In [ ]:
svm_roc = roc_auc_score(labels_test, svm_probs)
fpr, tpr, thresholds = roc_curve(labels_test, clf.predict_proba(features_test)[:,1])
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

plt.plot(fpr, tpr, label='SVM Polynomial Kernel (AUC = %0.3f)' % svm_roc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('Receiver operating characteristic', fontsize=15)
plt.legend(loc="lower right", fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_POLY_ROC.png', bbox_inches='tight')

cm = confusion_matrix(labels_test, svm_pred)
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

sns.heatmap(cm, cmap='Purples', annot=True, fmt='d', cbar=False, annot_kws = {"size": 15})
plt.title('SVM Sigmoid Kernel \nAccuracy:{0:.3f}'.format(accuracy_score(labels_test, svm_pred)), fontsize=15)
plt.ylabel('True label', fontsize=15)
plt.xlabel('Predicted label', fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_POLY_CM.png', bbox_inches='tight')

#### SVM with Sigmoid Kernel

In [ ]:
clf = OneVsRestClassifier(SVC(kernel='sigmoid', probability=True, class_weight=None), n_jobs=-1)
clf.fit(features_train, labels_train)
svm_pred = clf.predict(features_test)
svm_probs = clf.predict_proba(features_test)[:, 1]
print("SVM RBF Accuracy:",metrics.accuracy_score(labels_test, svm_pred))

In [ ]:
svm_roc = roc_auc_score(labels_test, svm_probs)
fpr, tpr, thresholds = roc_curve(labels_test, clf.predict_proba(features_test)[:,1])
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

plt.plot(fpr, tpr, label='SVM Polynomial Kernel (AUC = %0.3f)' % svm_roc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('Receiver operating characteristic', fontsize=15)
plt.legend(loc="lower right", fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_SIGMOID_ROC.png', bbox_inches='tight')

cm = confusion_matrix(labels_test, svm_pred)
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

sns.heatmap(cm, cmap='Purples', annot=True, fmt='d', cbar=False, annot_kws = {"size": 15})
plt.title('SVM Sigmoid Kernel \nAccuracy:{0:.3f}'.format(accuracy_score(labels_test, svm_pred)), fontsize=15)
plt.ylabel('True label', fontsize=15)
plt.xlabel('Predicted label', fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/SVM_SIGMOID_CM.png', bbox_inches='tight')

### XGBoost

In [ ]:
from xgboost import XGBClassifier

eval_set=[(features_test, labels_test)]
xgb = XGBClassifier()
xgb.fit(features_train, labels_train, eval_metric='error', eval_set=eval_set, verbose=True, early_stopping_rounds=25)
xgb_probs = xgb.predict_proba(features_test)[:, 1]
xgb_pred = xgb.predict(features_test)
print("Accuracy:",metrics.accuracy_score(labels_test, xgb_pred))

In [ ]:
xgb_roc = roc_auc_score(labels_test, xgb_probs)
fpr, tpr, thresholds = roc_curve(labels_test, xgb_probs)
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

plt.plot(fpr, tpr, label='XGBoost (area = %0.3f)' % xgb_roc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('Receiver operating characteristic', fontsize=15)
plt.legend(loc="lower right", fontsize=15)
plt.show()
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/XGB_ROC.png', bbox_inches='tight')

cm = confusion_matrix(labels_test, xgb_pred)
plt.figure(figsize=(16,12))
sns.set_style("whitegrid")

sns.heatmap(cm, cmap='Purples', annot=True, fmt='d', cbar=False, annot_kws = {"size": 15})
plt.title('XGBoost \nAccuracy:{0:.3f}'.format(accuracy_score(labels_test, xgb_pred)), fontsize=15)
plt.ylabel('True label', fontsize=15)
plt.xlabel('Predicted label', fontsize=15)
plt.savefig('/gpfs/scratch/bcc276/EHR_Dream/Figures/XGB_CM.png', bbox_inches='tight')